In [0]:
# CISC642 PR3 - Matthew Leinhauser

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [2]:
# create the training and test sets

dataset_root = './data'

# resize the images to be the same size, convert them to be Tensor Objects, and
# normalize the tensor values based on the mean and standard deviation of the
# RGB values

data_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5],
                                                           [0.5, 0.5, 0.5])])

trainset = torchvision.datasets.CIFAR100(dataset_root, train=True, transform =
                                         data_transforms, download=True)
testset = torchvision.datasets.CIFAR100(dataset_root, train=False, transform =
                                        data_transforms, download=True)

# create a data loader for both the training set and testing set
train_loader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size =16, shuffle=False)

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# load the VGG16 model wiith pretrained weights
model = models.vgg16(pretrained=True)

# extract the number of input features for the last fully connected model layer
num_features = model.classifier[6].in_features

# extract the first layer of the model
num_class = model.classifier[0].in_features

# replace the last fully connected layer with a new layer
model.classifier[6] = nn.Linear(num_features, num_class)

# freeze all layers except the last layer (output) because we don't want to
# update them
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier[6].parameters():
  param.requires_grad = True

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [0]:
# hyperparameters
num_epochs = 10
learning_rate = 0.001

# set up the model for training
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [5]:
# TRAINING THE MODEL

# iterate over the mini-batchs and set the optimizer as zero_grad(). Then use
# the current model weight for predication and backpropogating the prediction
# loss

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward Pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Optimize Backward Pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Track the accuracy and print out where we are in the training process
    total = labels.size(0)
    _, preds = torch.max(outputs, 1)
    correct = (preds == labels).sum().item()
    if(i + 1) % 1000 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
      .format(epoch + 1, num_epochs, i + 1, len(train_loader), loss.item(), 
              (correct/ total) * 100))


# Save the model weights
model_weights = copy.deepcopy(model.state_dict())
torch.save(model_weights, 'best_model_weight.pth')

Epoch [1/10], Step [1000/3125], Loss: 2.4299, Accuracy: 31.25%
Epoch [1/10], Step [2000/3125], Loss: 2.0785, Accuracy: 31.25%
Epoch [1/10], Step [3000/3125], Loss: 2.4462, Accuracy: 43.75%
Epoch [2/10], Step [1000/3125], Loss: 2.4374, Accuracy: 43.75%
Epoch [2/10], Step [2000/3125], Loss: 1.5918, Accuracy: 56.25%
Epoch [2/10], Step [3000/3125], Loss: 1.4399, Accuracy: 56.25%
Epoch [3/10], Step [1000/3125], Loss: 1.8626, Accuracy: 43.75%
Epoch [3/10], Step [2000/3125], Loss: 2.0511, Accuracy: 37.50%
Epoch [3/10], Step [3000/3125], Loss: 1.1466, Accuracy: 68.75%
Epoch [4/10], Step [1000/3125], Loss: 1.8722, Accuracy: 56.25%
Epoch [4/10], Step [2000/3125], Loss: 1.1630, Accuracy: 68.75%
Epoch [4/10], Step [3000/3125], Loss: 2.0740, Accuracy: 56.25%
Epoch [5/10], Step [1000/3125], Loss: 1.6591, Accuracy: 56.25%
Epoch [5/10], Step [2000/3125], Loss: 1.6481, Accuracy: 62.50%
Epoch [5/10], Step [3000/3125], Loss: 2.4332, Accuracy: 18.75%
Epoch [6/10], Step [1000/3125], Loss: 2.3256, Accuracy:

In [7]:
# TESTING THE MODEL
num_correct = 0
num_samples = 0

model.load_state_dict(torch.load('best_model_weight.pth'))

model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    num_samples += labels.size(0)
    num_correct += (preds == labels).sum().item()
  
print('Accuracy: ', (num_correct/num_samples) * 100, '%')

Accuracy:  57.85 %


# **PART 2**

In [0]:
# Create the model
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 20, 5, 1, 2),
        nn.ReLU(),
        nn.MaxPool2d(2, 4))
    self.layer2 = nn.Sequential(
        nn.Conv2d(20, 40, 5, 1, 2),
        nn.ReLU(),
        nn.MaxPool2d(2, 2))
    self.layer3 = nn.Sequential(
        nn.Conv2d(40, 80, 5, 1, 2),
        nn.ReLU(),
        nn.MaxPool2d(2, 2))
    # add a dropout layer to avoid overfitting
    self.drop_out = nn.Dropout()
    self.fully_con_1 = nn.Linear(14 * 14 * 80, 200)
    self.fully_con_2 = nn.Linear(200, 100)

  def forward(self, x):
    output = self.layer1(x)
    output = self.layer2(output)
    output = self.layer3(output)
    output = output.reshape(output.size(0), -1)
    output = self.drop_out(output)
    output = self.fully_con_1(output)
    output = self.fully_con_2(output)
    return output


In [0]:
# Set up the model

# hyperparameters
num_epochs = 30
learning_rate = 0.001

# set up the model for training
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [20]:
# Train the model
# iterate over the mini-batchs and set the optimizer as zero_grad(). Then use
# the current model weight for predication and backpropogating the prediction
# loss

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward Pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Optimize Backward Pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Track the accuracy and print out where we are in the training process
    total = labels.size(0)
    _, preds = torch.max(outputs, 1)
    correct = (preds == labels).sum().item()
    if(i + 1) % 1000 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
      .format(epoch + 1, num_epochs, i + 1, len(train_loader), loss.item(), 
              (correct/ total) * 100))

# Save the model weights
model_weights = copy.deepcopy(model.state_dict())
torch.save(model_weights, 'best_model_weight.pth')

Epoch [1/30], Step [1000/3125], Loss: 4.3576, Accuracy: 0.00%
Epoch [1/30], Step [2000/3125], Loss: 4.3636, Accuracy: 0.00%
Epoch [1/30], Step [3000/3125], Loss: 3.4498, Accuracy: 18.75%
Epoch [2/30], Step [1000/3125], Loss: 3.2536, Accuracy: 12.50%
Epoch [2/30], Step [2000/3125], Loss: 3.5796, Accuracy: 25.00%
Epoch [2/30], Step [3000/3125], Loss: 2.7496, Accuracy: 43.75%
Epoch [3/30], Step [1000/3125], Loss: 2.8295, Accuracy: 25.00%
Epoch [3/30], Step [2000/3125], Loss: 2.1158, Accuracy: 37.50%
Epoch [3/30], Step [3000/3125], Loss: 2.2985, Accuracy: 37.50%
Epoch [4/30], Step [1000/3125], Loss: 1.3366, Accuracy: 68.75%
Epoch [4/30], Step [2000/3125], Loss: 1.7477, Accuracy: 56.25%
Epoch [4/30], Step [3000/3125], Loss: 2.5659, Accuracy: 31.25%
Epoch [5/30], Step [1000/3125], Loss: 3.4999, Accuracy: 6.25%
Epoch [5/30], Step [2000/3125], Loss: 2.0580, Accuracy: 50.00%
Epoch [5/30], Step [3000/3125], Loss: 2.0773, Accuracy: 43.75%
Epoch [6/30], Step [1000/3125], Loss: 1.7321, Accuracy: 43

In [21]:
# Test the model
num_correct = 0
num_samples = 0

model.load_state_dict(torch.load('best_model_weight.pth'))

model.eval()
for epoch in range(num_epochs):
# with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    num_samples += labels.size(0)
    num_correct += (preds == labels).sum().item()
  
print('Accuracy: ', (num_correct/num_samples) * 100, '%')

Accuracy:  40.510000000000005 %
